In [1]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
targets_path=r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Accelerometer_Data\Datasets\HAR_Dataset\targets.npy"
dataset_path=r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Accelerometer_Data\Datasets\HAR_Dataset\data.npy"


In [ ]:
def clean_taxonomy(behaviour):
    
    if("AG Or Ma" in behaviour):
        return "AG Or Ma"
    
    if("AG Or Mi" in behaviour):
        if("Ag" in behaviour):
            return "AG Or Mi Ag"
        else:
            return behaviour
        
    if("AG Cp" in behaviour):
        if(not("Ds" in behaviour)):
            return "AG Cp Null"
        else:
            return behaviour
    else: 
        return behaviour

In [3]:
targets=np.load(targets_path,allow_pickle=True)
data=np.load(dataset_path,allow_pickle=True)

In [11]:
next(target for target in targets if "Null" in target)

'Null'

In [ ]:
targets_df=pd.Series(targets,name="targets")

In [ ]:
targets_df.value_counts()

In [ ]:
targets=list(map(lambda target: clean_taxonomy(target),targets))
targets=np.asarray(targets)

In [12]:
from sklearn.model_selection import train_test_split
#Train/val/test
def split_data(data,targets,test_size):
    train_idx,test_idx=train_test_split(range(len(data)),test_size=test_size,random_state=42)
    samples_train=data[train_idx]
    targets_train=targets[train_idx]

    samples_test=data[test_idx]
    targets_test=targets[test_idx]
    
    return samples_train,targets_train,samples_test,targets_test

def save_data(folder,data,targets):
    np.save(os.path.join(folder,"data"),data)
    np.save(os.path.join(folder,"targets"),targets)

In [13]:
samples_train,targets_train,samples_test,targets_test=split_data(data,targets,0.2)
samples_train,targets_train,samples_val,targets_val=split_data(samples_train,targets_train,0.1)

In [14]:
train_folder=r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Accelerometer_Data\Datasets\HAR_Dataset_raw\Train"
test_folder=r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Accelerometer_Data\Datasets\HAR_Dataset_raw\Test"
val_folder=r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Accelerometer_Data\Datasets\HAR_Dataset_raw\Val"

In [15]:
save_data(train_folder,samples_train,targets_train)
save_data(test_folder,samples_test,targets_test)
save_data(val_folder,samples_val,targets_val)

# splitting data

In [2]:
participants_dir = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\FinalDatasets\Participants"

In [14]:
participant_folders = [os.path.join(participants_dir,participant) for participant in os.listdir(participants_dir)]

In [24]:
participants_train = participant_folders[:25]
participants_val  = participant_folders[25:30]
participants_test  = participant_folders[-5:]

In [25]:
participants_test

['C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\FinalDatasets\\Participants\\2904d',
 'C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\FinalDatasets\\Participants\\3004a',
 'C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\FinalDatasets\\Participants\\3004b',
 'C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\FinalDatasets\\Participants\\3004c',
 'C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\FinalDatasets\\Participants\\3004e']

In [33]:
def get_data(data_folders,subset,dataset_dir):
    segments =[]
    labels = []
    for folder in data_folders: 
        segments_file = os.path.join(folder,"FullDataset","segments_data.npy")
        labels_file =  os.path.join(folder,"FullDataset","targets_data.npy")
        segments.append(np.load(segments_file))
        labels.append(np.load(labels_file))
    segments = np.concatenate(segments)
    labels = np.concatenate(labels)
    
    np.save(os.path.join(dataset_dir,subset,"data"),segments)
    np.save(os.path.join(dataset_dir,subset,"targets"),labels)

In [32]:
dataset_path = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\FinalDatasets\Datasets\HAR_dataset_v1"

In [34]:
folders  = [participants_train,participants_test,participants_val]
subsets  =["Train","Test","Val"]
for folders,subset in zip(folders,subsets):
    get_data(folders,subset,dataset_path)

In [ ]:
#concat every subtable for each participant
participant_folders=[os.path.join(participant_folder,folder) for folder in os.listdir(participant_folder)]

for participant_folder in tqdm(participant_folders):  
    folder=os.path.join(participant_folder,data_type)
    table_arr=[]
    for table in os.listdir(folder):
        table_arr.append(pd.read_csv(os.path.join(folder,table)))
    df=pd.concat(table_arr)
    participant=os.path.basename(participant_folder)
    df.to_csv(os.path.join(folder,"{0}_raw_{1}.csv".format(data_type,participant)),index=False)
    print("Save table of participant : {}".format(participant))
